In [5]:
import robotic as ry

C = ry.Config()
from raiSimulationEnv import *

In [9]:
C = ry.Config()
C.addFile("rai-robotModels/scenarios/pandasFight.g")
C.setJointState(np.array([ 0.  , -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.  ,  0.  ,
       -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.]))



initialFrameState = C.getFrameState()
qhome = C.getJointState()
q0 = qhome
armed = np.load('armed.npy')


def initializeSimulation(render=True):
    C.setFrameState(initialFrameState)
    S = ry.Simulation(C, ry.SimulationEngine.physx, verbose=0)
    if render:
        C.view()
    simulationGoTo(S,armed,render=render)
    simulationCloseGrippers(S,render=render)
    simulationGoHome(S,render=render)
    return S

initializeSimulation()

KeyboardInterrupt: 

In [11]:
import gymnasium as gym
import numpy as np
import time
import robotic as ry

C = ry.Config()
C.addFile("rai-robotModels/scenarios/pandasFight.g")
C.setJointState(np.array([ 0.  , -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.  ,  0.  ,
       -1.  ,  0.  , -2.  ,  0.  ,  2.  ,  0.]))



initialFrameState = C.getFrameState()
qhome = C.getJointState()
q0 = qhome
armed = np.load('armed.npy')

def simulationCloseGrippers(S, tau=0.01, render=True):
    S.closeGripper('l_gripper', width=0.0001, speed=1)
    while (not S.getGripperIsGrasping('l_gripper')) and (S.getGripperWidth('l_gripper') > 0.001):
        S.step([], tau, ry.ControlMode.none)
        if render:
            C.view()
            time.sleep(tau)

    S.closeGripper('r_gripper', width=0.0001, speed=1)
    while (not S.getGripperIsGrasping('r_gripper')) and (S.getGripperWidth('r_gripper') > 0.001):
        S.step([], tau, ry.ControlMode.none)
        if render:
            C.view()
            time.sleep(tau)

def simulationOpenGrippers(S, tau=0.01, render=True):
    S.openGripper('l_gripper', width=0.05, speed=0.5)
    while S.getGripperWidth('l_gripper') < 0.05 - 0.01:
        S.step([], tau, ry.ControlMode.none)
        if render:
            C.view()
            time.sleep(tau)
    S.openGripper('r_gripper', width=0.05, speed=0.5)
    while S.getGripperWidth('r_gripper') < 0.05 - 0.01:
        S.step([], tau, ry.ControlMode.none)
        if render:
            C.view()
            time.sleep(tau)

def simulationGoTo(S, q, tau=0.01, checkCol=False, render=True):
    checkColTime = 0.1
    timer = 0
    while np.linalg.norm(S.get_q() - q) > 0.01:
        S.step(q, tau, ry.ControlMode.position)
        timer += tau
        if render:
            C.view()
            time.sleep(tau)

        if timer > 2:
            print("Target cannot be reached within 2 seconds.")
            return "failedReach"
        if checkCol:
            if timer > checkColTime:
                if findCollision(C, 'sword_1'):
                    return "collision"
                else:
                    checkColTime += 0.1

def simulationGoHome(S, tau=0.01, render=True):
    checkColTime = 0.1
    timer = 0
    while np.linalg.norm(S.get_q() - q0) > 0.01:
        S.step(q0, tau, ry.ControlMode.position)
        timer += tau
        if render:
            C.view()
            time.sleep(tau)
        # if timer > checkColTime:
        #     if findCollision(C, 'sword_1'):
        #         break
        #     else:
        #         checkColTime += 0.1
            
def simulationFollowPath(S, path, tau=0.01, render=True):
    checkColTime = 0.1
    timer = 0
    for i in range(len(path)):
        while np.linalg.norm(S.get_q() - path[i]) > 0.3:
            print(i)
            print(np.linalg.norm(S.get_q() - path[i]))
            S.step(path[i], tau, ry.ControlMode.position)
            timer += tau
            if render:
                C.view()
                time.sleep(tau)
            if timer > checkColTime:
                print('check_col')
                if findCollision(C, 'sword_1'):
                    return
                else:
                    checkColTime += 0.1

    simulationGoTo(S, path[-1], checkCol=True,render=render)

def simulationWait(S, t, tau=0.01, render=True):
    for k in range(int(t / tau)):
        S.step([], tau, ry.ControlMode.none)
        if render:
            C.view()
            time.sleep(tau)

def simulationTorqueCtrl(S, t, torque, tau=0.01, render=True):
    for k in range(int(t / tau)):
        S.step(torque, tau, ry.ControlMode.acceleration)
        if render:
            C.view()
            time.sleep(tau)

def simulationVelocityCtrl(S,t,vel,tau=0.01, render=True):
    for k in range(int(t/tau)):
        S.step(vel,tau,ry.ControlMode.velocity)
        if render:
            C.view()
            time.sleep(tau)

def followInterpolatedPath(S,path,tau=0.01, render=True):
    checkColTime = 0.1
    timer = 0
    for i in range(len(path)):
        print(i)
        print(np.linalg.norm(S.get_q()- path[i]))
        S.step(path[i],tau,ry.ControlMode.position)
        if render:
            C.view()
            time.sleep(tau)
        timer += tau
        if timer > checkColTime:
            if findCollision(C,'sword_1'):
                return
            else:
                checkColTime += 0.1
    simulationGoTo(S,path[-1],checkCol=True,render=render)

def findCollision(C,object1):
    collisions = [col for col in C.getCollisions(0) if object1 in col and not col[1].startswith('l_') and not col[0].startswith('l_')]
    if len(collisions) > 0:
        return True
    else:
        return False
    
def findRewardingCollision(C,object1):
    collisions = [col for col in C.getCollisions(0) if object1 in col and col[1].startswith('r_') or col[0].startswith('r_')]
    if len(collisions) > 0:
        return True
    else:
        return False
    
    
# Use this code. It is awesome. It will follow the path and stop if there is a collision.    
def followSplinePath(S,path,t,tau=0.01, render=True):
    """
        Guides a simulation object along a specified spline path.
        Parameters:
        S (Simulation): The simulation object that will follow the path.
        path (array): A list of waypoints defining the spline path.
        t (float): The total time duration for following the path.
        tau (float, optional): The time step for each simulation step. Default is 0.01.
        The function performs the following steps:
        1. Resets the spline reference of the simulation object.
        2. Sets the spline reference with the given path and a time vector.
        3. Iteratively steps through the simulation, updating the view and checking for collisions.
        4. If a collision with 'sword_1' is detected, the function exits early.
        5. The loop runs for a duration slightly longer than `t` to ensure completion.
        Note:
        - The `+20` in the loop range is a heuristic to prevent early termination when `t` is low.
        - The function checks for collisions every 0.1 seconds.

    """
    checkColTime = 0.1
    timer = 0
    S.resetSplineRef() # Reset previous spline reference
    S.setSplineRef(path,np.linspace(0.01,t,len(path))) # Set new spline reference
    joint_data = np.empty((int(t/tau)+20, 2,S.get_q().shape[0])) # Initialize array to store joint data
    for k in range(int(t/tau)+20): # This +20 is heuristic. It was stopping after a short time when t was low.
        S.step([],tau,ry.ControlMode.spline)
        timer += tau
        if render:
            C.view()
            time.sleep(tau)
        inst_pos = S.get_q()
        inst_vel = S.get_qDot()
        joint_data[k,:,:] = [inst_pos,inst_vel]
        if timer > checkColTime: # Check for collisions every 0.1 seconds
            if findCollision(C,'sword_1'): # Check if sword is colliding with something (left robot is excluded (no self collision))
                return joint_data[:k,:,:][np.newaxis,:,:,:] # Return the joint data until collision
            else:
                checkColTime += 0.1 # Increment the collision check time

    return joint_data[np.newaxis,:,:,:]
    #simulationGoTo(S,path[-1],checkCol=True) 

def initializeSimulation(render=True):
    C.setFrameState(initialFrameState)
    S = ry.Simulation(C, ry.SimulationEngine.physx, verbose=0)
    if render:
        C.view()
    simulationGoTo(S,armed,render=render)
    simulationCloseGrippers(S,render=render)
    simulationGoHome(S,render=render)
    return S

In [12]:
initializeSimulation()

-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-sword_0 of type rigid with rel [0, 0, 0]
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT r_panda_joint7-shi of type rigid with rel [0, 0, 0]


In [13]:
C.getFrameNames()

['world',
 'origin',
 'table',
 'l_panda_base',
 'l_panda_link0',
 'l_panda_link0_0',
 'l_panda_joint1_origin',
 'l_panda_joint1',
 'l_panda_link1',
 'l_panda_link1_0',
 'l_panda_joint2_origin',
 'l_panda_joint2',
 'l_panda_link2',
 'l_panda_link2_0',
 'l_panda_joint3_origin',
 'l_panda_joint3',
 'l_panda_link3',
 'l_panda_link3_0',
 'l_panda_joint4_origin',
 'l_panda_joint4',
 'l_panda_link4',
 'l_panda_link4_0',
 'l_panda_joint5_origin',
 'l_panda_joint5',
 'l_panda_link5',
 'l_panda_link5_0',
 'l_panda_joint6_origin',
 'l_panda_joint6',
 'l_panda_link6',
 'l_panda_link6_0',
 'l_panda_joint7_origin',
 'l_panda_joint7',
 'l_panda_link7',
 'l_panda_link7_0',
 'l_panda_joint8_origin',
 'l_panda_joint8',
 'l_panda_link8',
 'l_panda_hand_joint_origin',
 'l_panda_hand_joint',
 'l_panda_hand',
 'l_panda_hand_0',
 'l_panda_finger_joint1_origin',
 'l_panda_finger_joint2_origin',
 'l_panda_finger_joint1',
 'l_panda_finger_joint2',
 'l_panda_leftfinger',
 'l_panda_rightfinger',
 'l_panda_leftfi

In [32]:
ry.params_print()

physx/verbose: 0,
physx/yGravity!,
physx/softBody!,
physx/multiBody,
physx/multiBodyDisableGravity,
physx/jointedBodies!,
physx/angularDamping: 0.1,
physx/defaultFriction: 1,
physx/defaultRestitution: 0.1,
physx/motorKp: 1000,
physx/motorKd: 100

In [21]:
del S

In [27]:
ry.params_clear()

In [28]:
ry.params_print()

In [ ]:
from raiSimulationEnv import *
from ddpgCode.evaluator import Evaluator

def load_data(folder):
    datas = []
    for path in os.listdir(folder):
        if path.endswith('.npy'):
            path = np.load(os.path.join(folder, path)).reshape(-1,28)
            datas.append(path)
    return datas


env = RobotSimEnv()
env.reset()

evaluator = Evaluator()
